In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_retrieval

## Setup
In this notebook, we will run SfM reconstruction from scratch on a set of images. We choose the [South-Building dataset](https://openaccess.thecvf.com/content_cvpr_2013/html/Hane_Joint_3D_Scene_2013_CVPR_paper.html) - we will download it later. First, we define some paths.

In [4]:
dataset = Path('datasets/COMP90086')
images = dataset / 'images/'

outputs = Path('outputs/COMP90086/')
sfm_pairs = outputs / 'pairs-manual-db-covis5.txt'
sfm_dir = outputs / 'sfm_superpoint+superglue'

feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

## Download the dataset
The dataset is simply a set of images. The intrinsic parameters will be extracted from the EXIF data, and refined with SfM.

## Extract local features for database and query images

In [4]:
feature_path = extract_features.main(feature_conf, images, outputs)

[10/16/2021 06:08:05 INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}
[10/16/2021 06:08:05 INFO] Found 8700 images in root datasets/COMP90086/images.
Loaded SuperPoint model


100%|██████████| 8700/8700 [1:24:27<00:00,  1.72it/s]

[10/16/2021 07:32:32 INFO] Finished exporting features.


## Matching



In [5]:
match_path = match_features.main(
    matcher_conf, sfm_pairs, feature_conf['output'], outputs, exhaustive=False)

[10/16/2021 07:39:06 INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}
Loaded SuperGlue model ("outdoor" weights)


100%|██████████| 30000/30000 [5:53:25<00:00,  1.41it/s]  

[10/16/2021 13:32:36 INFO] Finished exporting matches.


In [8]:
feature_path

PosixPath('outputs/COMP90086/feats-superpoint-n4096-r1024.h5')

## SfM reconstruction
Run COLMAP on the features and matches.

In [28]:
feature_path = outputs / 'feats-superpoint-n4096-r1024.h5'
match_path = outputs / 'feats-superpoint-n4096-r1024_matches-superglue_pairs-manual-db-covis5.h5'

reconstruction.main(sfm_dir, images / 'train', sfm_pairs, feature_path, match_path, skip_geometric_verification = True)

[10/16/2021 16:29:49 INFO] Creating an empty database...
[10/16/2021 16:29:49 INFO] Importing images into the database...

Feature import

Processing file [1/7500]
  Features:       0
Processing file [2/7500]
  Features:       0
Processing file [3/7500]
  Features:       0
Processing file [4/7500]
  Features:       0
Processing file [5/7500]
  Features:       0
Processing file [6/7500]
  Features:       0
Processing file [7/7500]
  Features:       0
Processing file [8/7500]
  Features:       0
Processing file [9/7500]
  Features:       0
Processing file [10/7500]
  Features:       0
Processing file [11/7500]
  Features:       0
Processing file [12/7500]
  Features:       0
Processing file [13/7500]
  Features:       0
Processing file [14/7500]
  Features:       0
Processing file [15/7500]
  Features:       0
Processing file [16/7500]
  Features:       0
Processing file [17/7500]
  Features:       0
Processing file [18/7500]
  Features:       0
Processing file [19/7500]
  Features:     

100%|██████████| 7500/7500 [00:02<00:00, 2741.97it/s]


[10/16/2021 16:30:12 INFO] Importing matches into the database...


  0%|          | 0/30000 [00:00<?, ?it/s]


IntegrityError: UNIQUE constraint failed: matches.pair_id

In [ ]:
# Run matches.importer manually

# colmap matches_importer --database_path outputs/COMP90086/sfm_superpoint+superglue/database.db --match_list_path outputs/COMP90086/pairs-manual-db-covis5.txt --match_type pairs --SiftMatching.max_num_trials 20000 --SiftMatching.min_inlier_ratio 0.1 --SiftMatching.use_gpu 0

## Visualization
We visualize some of the registered images, and color their keypoint by visibility, track length, or triangulated depth.

In [25]:
visualization.visualize_sfm_2d(sfm_dir, images, color_by='visibility', n=5)

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/COMP90086/sfm_superpoint+superglue/images.bin'

In [ ]:
visualization.visualize_sfm_2d(sfm_dir, images, color_by='track_length', n=5)

In [ ]:
visualization.visualize_sfm_2d(sfm_dir, images, color_by='depth', n=5)

# TODO: When querying

## Generate pairs for the SfM reconstruction

Instead of matching all database images exhaustively, we exploit the existing SIFT model to find which image pairs are the most covisible. 
We first convert the SIFT model from the NVM to the COLMAP format, and then do a covisiblity search, selecting the top 20 most covisibile neighbors for each image.

In [ ]:
num_covis = 20
# top-k most covisible in SIFT model
sfm_pairs = outputs / f'pairs-db-covis{num_covis}.txt'

sift_sfm = dataset / '3D-models/aachen_v_1_1'

In [ ]:
pairs_from_covisibility.main(
    sift_sfm, sfm_pairs, num_matched=20)


In [ ]:
sfm_matches = match_features.main(
    matcher_conf, sfm_pairs, feature_conf['output'], outputs)

## Match the database images

The function returns the path of the file in which all the computed matches are stored

In [ ]:
#sfm_match_path = match_features.main(matcher_conf, sfm_pairs, feature_conf['output'], outputs)

## Match the query images

We try to find matches for the given images by using `hloc/pairs_from_retrieval.py`.

In [9]:
# Output top 50 matches
netvlad_top_k = 50


# Output file with pairs
# top-k retrieved by NetVLAD
loc_pairs = outputs / f'pairs-query-netvlad{netvlad_top_k}.txt'  

In [10]:
# Query images have 'test' prefix, db images have 'train' prefix
pairs_from_retrieval.main(
    feature_path, loc_pairs, netvlad_top_k,
    query_prefix='test', db_prefix = 'train', db_model=sfm_dir)

[10/16/2021 14:29:04 INFO] Extracting image pairs from a retrieval database.


FileNotFoundError: [Errno 2] No such file or directory: 'outputs/COMP90086/sfm_superpoint+superglue/images.bin'

In [ ]:
match_path = match_features.main(
    matcher_conf, loc_pairs, feature_conf['output'], outputs)

# Localize the query images

We finally perform localization via `hloc.localize_sfm.py`

In [13]:
results = outputs / f'COMP90086_hloc_superpoint+superglue_netvlad{netvlad_top_k}.txt'

NameError: name 'netvlad_top_k' is not defined

In [ ]:
localize_sfm.main(
    sfm_dir, #Done
    outputs / 'query_list_with_intrinsics.txt', #DONE
    loc_pairs, # DONE
    feature_path, #DONE
    match_path, #DONE
    results, # Done
    covisibility_clustering=False)  # not required with SuperPoint+SuperGlue